<img src="images/header.png" alt="Logo UCLM-ESII" align="right">

<br><br><br><br>
<h2><font color="#92002A" size=4>Trabajo Fin de Grado</font></h2>

<h1><font color="#6B001F" size=5>Generación automática de playlist de canciones <br> mediante técnicas de minería de datos</font></h1>
<h2><font color="#92002A" size=3>Parte 9 - Preprocesamiento 2</font></h2>

<br>
<div style="text-align: right">
    <font color="#B20033" size=3><strong>Autor</strong>: <em>Miguel Ángel Cantero Víllora</em></font><br>
    <br>
    <font color="#B20033" size=3><strong>Directores</strong>: <em>José Antonio Gámez Martín</em></font><br>
    <font color="#B20033" size=3><em>Juan Ángel Aledo Sánchez</em></font><br>
    <br>
<font color="#B20033" size=3>Grado en Ingeniería Informática</font><br>
<font color="#B20033" size=2>Escuela Superior de Ingeniería Informática | Universidad de Castilla-La Mancha</font>

</div>

---

<br>


<a id="indice"></a>
<h2><font color="#92002A" size=5>Índice</font></h2>

<br>

* [1. Introducción](#section1)
* [2. Creación de matrices dispersas](#section2)
    * [2.1. Pistas](#section21)
    * [2.2. Artistas](#section22)
* [3. Creación de características de playlists](#section3)
* [4. Almacenamiento del DataSet en formato LightFM](#section4)


<br>

---

In [1]:
import csv
import os
import pickle
import joblib
import scipy.sparse as sp

from lightfm.data import Dataset
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelBinarizer

In [2]:
# Variables globales

# Directorio empleado para guardar/leer los datos generados
MPD_CSV_PATH = 'MPD_CSV'

ALBUMS_FILE = os.path.join(MPD_CSV_PATH,'mpd.albums.csv')
ARTISTS_FILE = os.path.join(MPD_CSV_PATH,'mpd.artists.csv')
TRACKS_FILE = os.path.join(MPD_CSV_PATH,'mpd.tracks.csv')
PLSTRS_FILE = os.path.join(MPD_CSV_PATH,'mpd.pls-tracks.csv')
PLSTARTISTS_FILE = os.path.join(MPD_CSV_PATH,'mpd.pls-artists.csv')
PLSINFO_FILE = os.path.join(MPD_CSV_PATH,'mpd.playlists-info.csv')
PLSTESTINFO_FILE = os.path.join(MPD_CSV_PATH,'mpd.playlists-info-test.csv')
NAMES_FILE = os.path.join(MPD_CSV_PATH, "mpd.names.csv")

MPD_LFM_FILE = os.path.join(MPD_CSV_PATH, "mpd_lightfm.pickle")

---

<br>


<a id="section1"></a>
## <font color="#92002A">1 - Introducción</font>
<br>

**LightFM** requiere que los datos que se le pasen al modelo sean matrices de expansión. En el caso de las características de usuario serán matrices *CSR* (matrices de expansión comprimidas), mientras que las interacciones entre usuarios e items serán matrices *COO* (matriz de expansión en formato de coordenadas). En caso de que las interacciones entre usuario e item contengan un peso, también se requerirá que estas sean una matriz COO.

Para facilitarnos esta tarea, en la biblioteca de *LightFM* podemos encontrar la clase `lightfm.data.Dataset`, la cual proporciona una serie de funciones para construir las matrices de interacción y las de características.

<br>

---

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section2"></a>
## <font color="#92002A">2 - Creación de matrices de interacción</font>
<br>




<br>

<a id="section21"></a>
### <font color="#92002A">2.1 - Pistas</font>

<br>

In [3]:
def get_plsartists():
    file = open(PLSTARTISTS_FILE, "r")
    return csv.DictReader((line for line in file))

def get_plstrs():
    file = open(PLSTRS_FILE, "r")
    return csv.DictReader((line for line in file))

def get_artists():
    file = open(ARTISTS_FILE, "r", encoding='utf8')
    return csv.DictReader((line for line in file))

def get_tracks():
    file = open(TRACKS_FILE, "r", encoding='utf8')
    return csv.DictReader((line for line in file))

def get_names():
    file = open(NAMES_FILE, "r", encoding='utf8')
    return csv.DictReader((line for line in file))

In [4]:
mpd_tracks = Dataset()

In [5]:
mpd_tracks.fit(users=(x['pl_pid'] for x in get_names()),
               items=(x['track_pid'] for x in get_tracks()))

In [6]:
num_pls, num_trs = mpd_tracks.interactions_shape()
print('num_pls: {}, num_trs {}.'.format(num_pls, num_trs))

num_pls: 1010000, num_trs 3759782.


In [7]:
tracks_interactions, _ = mpd_tracks.build_interactions(((x['pl_pid'], x['track_pid']))
                                                       for x in get_plstrs())
print(repr(tracks_interactions))

<1010000x3759782 sparse matrix of type '<class 'numpy.int32'>'
	with 91895507 stored elements in COOrdinate format>


---


<br>


<a id="section22"></a>
### <font color="#92002A">2.2 - Artistas</font>

<br>

In [8]:
mpd_artists = Dataset()

In [9]:
mpd_artists.fit(users=(x['pl_pid'] for x in get_names()),
                items=(x['artist_pid'] for x in get_artists()))

In [10]:
num_pls, num_artists = mpd_artists.interactions_shape()
print('num_pls: {}, num_artists {}.'.format(num_pls, num_artists))

num_pls: 1010000, num_artists 237518.


In [13]:
arts_interactions, arts_weights = mpd_artists.build_interactions(((x['pl_pid'], x['artist_pid'],
                                                                   int(x['artist_count']))
                                                                  for x in get_plsartists()))
print(repr(arts_interactions))
print(repr(arts_weights))

<1010000x237518 sparse matrix of type '<class 'numpy.int32'>'
	with 53768798 stored elements in COOrdinate format>
<1010000x237518 sparse matrix of type '<class 'numpy.float32'>'
	with 53768798 stored elements in COOrdinate format>


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section3"></a>
## <font color="#92002A">3 - Creación de características de playlists</font>
<br>

Para el caso de la matriz de características de las playlists hemos empleado la función de preprocesamiento ***LabelBinarizer***, que contiene la biblioteca de *scikit-learn*.

<br>

In [14]:
import scipy.sparse as sp

In [15]:
# Lectura de las etiquetas de playlists
pls_tags_list = [row['tags'] for row in get_names()]
pls_names_list = [row['name'] for row in get_names()]

# Transformación de etiquetas
labelbinarizer = LabelBinarizer(sparse_output=True)
plsfeatures_matrix = labelbinarizer.fit_transform(pls_names_list)
plsfeatures_matrix = sp.hstack([sp.eye(len(pls_names_list)), plsfeatures_matrix]) #Need to hstack user_features

In [16]:
plsfeatures_matrix

<1010000x1775995 sparse matrix of type '<class 'numpy.float64'>'
	with 2020000 stored elements in COOrdinate format>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section4"></a>
## <font color="#92002A">4 - Almacenamiento del DataSet en formato LightFM</font>
<br>

Una vez que hemos creado todas las matrices necesarias para trabajar con *LightFM*, las almacenamos en un diccionario y las guardamos en un archivo.

<br>

In [17]:
mpd_lfm_dict = {}

mpd_lfm_dict['plstrs_interactions'] = tracks_interactions
mpd_lfm_dict['artists_interactions'] = arts_interactions
mpd_lfm_dict['artists_weights'] = arts_weights
mpd_lfm_dict['pls_features'] = plsfeatures_matrix

In [18]:
if os.path.isfile(MPD_LFM_FILE):
    with open(MPD_LFM_FILE, "rb") as read_file:
        mpd_lfm_dict = pickle.load(read_file)
else:
    with open(MPD_LFM_FILE, "wb") as write_file:
        pickle.dump(mpd_lfm_dict, write_file, protocol=pickle.HIGHEST_PROTOCOL)

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-graduation-cap" aria-hidden="true" style="color:#92002A"></i> </font></div>